In [1]:
import re

from oauth2client.service_account import ServiceAccountCredentials
import gspread
import string


In [7]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds_sheet = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json', scope)
client = gspread.authorize(creds_sheet)

# Fetch PDF URLs from Google Sheets document
spreadsheet_key = '1xyQe3vgYKGl_hcPa1MOEn970xjTUirEUq_ewfiGjLRs'
sheet_name = 'Main'
worksheet = client.open_by_key(spreadsheet_key).worksheet(sheet_name)

In [8]:
texts = worksheet.col_values(163)
texts = [x for x in texts if x and x !='NOT_FOUND' and x !='error'] 
len(texts) 

44

In [9]:
texts

['3/6/23, 3:56 PM ﻋدد ﻛﺑﯾر ﻣن اﻟﻣﺣﺗﺟﯾن ﯾﺗﺟﻣﻌون اﻣﺎم ﻣﺳﺟد اﻻﻣﯾن ﻓﻲ وﺳط ﺑﯾروت .elnashra.com/news/show/1357503/ ﻋدد-ﻛﺑﯾر-اﻟﻣﺣﺗﺟﯾن-ﯾﺗﺟﻣﻌون-اﻣﺎم-ﻣﺳﺟد-اﻻﻣﯾن-وﺳط-ﺑﯾر 1/1 ﻋﺪد ﻛﺒﻴﺮ ﻣﻦ اﻟﻤﺤﺘﺠﻴﻦ ﻳﺘﺠﻤﻌﻮن اﻣﺎم ﻣﺴﺠﺪ اﻻﻣﻴﻦ ﻓﻲ وﺳﻂ ﺑﻴﺮوت ﻗﻀﺎء وأﻣﻦ12:16 2019 ﺗﺸﺮﻳﻦ اﻷول20 اﻷﺣﺪ وﺳﻂ ﺑﻴﺮوت ﻳﺘﺠﻤﻊ ﻋﺪد ﻛﺒﻴﺮ ﻣﻦ اﻟﻤﺤﺘﺠﻴﻦ ﻓﻲ ﻫﺬه ا��ﺛﻨﺎء اﻣﺎم ﻣﺴﺠﺪ ﻣﺤﻤﺪ ا��ﻣﻴﻦ ﻓﻲ ا��ﻣﻄﺎر راﻓﻌﻴﻦ ا��ﻋ��م اﻟﻠﺒﻨﺎﻧﻴﺔ وﻣﻜﺒﺮات اﻟﺼﻮت ﺗﺼﺪح ﺑﺎ��ﻏﺎﻧﻲ اﻟﻮﻃﻨﻴﺔ، وﻗﺪ ﺑﺪأت ﺑﺎﻟﻬﻄﻮل. \ue906 اﻟﻨﺸﺮة - أﺧﺒﺎر ﻟﺒﻨﺎن واﻟﺸﺮق اﻷوﺳﻂ',
 '3/8/23, 10:23 AM ﺟرﺣﻰ ﺗم ﻧﻘﻠﮭم ﻣن ﻣظﺎھرة اﻟﻧﺑطﯾﺔ إﻟﻰ اﻟﻣﺳﺗﺷﻔﻰ 5 :اﻟﺻﻠﯾب اﻷﺣﻣر .elnashra.com/news/show/1358505/ اﻟﺻﻠﯾب-اﻷﺣﻣر:-ﺟرﺣﻰ-ﻧﻘﻠﮭم-ﻣظﺎھرة-اﻟﻧﺑطﯾﺔ-إﻟﻰ-اﻟﻣﺳت 1/1 ﺟﺮﺣﻰ ﺗﻢ ﻧﻘﻠﻬﻢ ﻣﻦ ﻣﻈﺎﻫﺮة اﻟﻨﺒﻄﻴﺔ إﻟﻰ اﻟﻤﺴﺘﺸﻔﻰ5 :اﻟﺼﻠﻴﺐ اﻷﺣﻤﺮ ﻗﻀﺎء وأﻣﻦ18:11 2019 ﺗﺸﺮﻳﻦ اﻷول23 اﻷرﺑﻌﺎء 4 إﻟﻰ اﻟﻤﺴﺘﺸﻔﻰ و اﻟﻨﺒﻄﻴﺔ ﺟﺮﺣﻰ ﺗﻢ ﻧﻘﻠﻬﻢ ﻣﻦ ﻣﻈﺎﻫﺮة 5 أﻧﻪ اﻟﺼﻠﻴﺐ ا\x00\x00ﺣﻤﺮ اﻟﻠﺒﻨﺎﻧﻲ أﻋﻠﻦ ﺗﻤﺖ ﻣﻌﺎﻟﺠﺘﻬﻢ ﻓﻲ اﻟﻤﻜﺎن. ﺗﻤﻜﻨﺖ ﻣﻦ ﺗﻄﻮﻳﻖ ا\x00\x00ﺷﻜﺎل اﻟﺬي ﻗﻮى ا\x00\x00ﻣﻦ اﻟﺪاﺧﻠﻲ و اﻟﺠﻴﺶ اﻟﻠﺒﻨﺎﻧﻲ وﻛﺎﻧﺖ وﺣﺪات ﻣﻦ ﺑﻴﻦ اﻟﻤﺤﺘﺠﻴﻦ وﺷﺒﺎن وﺗﺴﺒﺐ ﺑﻮﻗﻮع ﻋﺪد ﻣﻦ اﻟ\x00\x00اي اﻟﺤﻜﻮﻣﻲ ﺣﺼﻞ

In [10]:
def clean_text(text):
    new_text = []
    for x in text.split():
        x = ''.join(i for i in x if i not in string.digits and i not in string.ascii_letters and i not in string.punctuation)
        x = x.replace('�','')
        new_text.append(x)
    return ' '.join(x for x in new_text if x)

In [11]:
# Clean the text in the column
count = 0
cleaned_values = []
for text in texts:
    cell_text = worksheet.find(text, )
    cleaned_text = clean_text(text)  # Remove non-Arabic characters
    worksheet.update_cell(cell_text.row, 172, cleaned_text)
    count += 1
    print((count * 100)/len(texts))

2.272727272727273
4.545454545454546
6.818181818181818
9.090909090909092
11.363636363636363
13.636363636363637
15.909090909090908
18.181818181818183
20.454545454545453
22.727272727272727
25.0
27.272727272727273
29.545454545454547
31.818181818181817
34.09090909090909
36.36363636363637
38.63636363636363
40.90909090909091
43.18181818181818
45.45454545454545
47.72727272727273
50.0
52.27272727272727
54.54545454545455
56.81818181818182
59.09090909090909
61.36363636363637
63.63636363636363
65.9090909090909
68.18181818181819
70.45454545454545
72.72727272727273
75.0
77.27272727272727
79.54545454545455
81.81818181818181
84.0909090909091
86.36363636363636
88.63636363636364
90.9090909090909
93.18181818181819
95.45454545454545
97.72727272727273
100.0
